In [1]:
from langchain_chroma import Chroma
from langchain_openai import AzureOpenAIEmbeddings
from config import *

In [3]:
embeddings=AzureOpenAIEmbeddings(
        azure_endpoint="https://ai-gpu-ps.openai.azure.com/",
        azure_deployment="gpt40-mini-long-context",
        openai_api_version="2024-05-01-preview",
        api_key="dc415207c54e4dd8ba8b60cb66374822"
    )

In [4]:
company_collection = Chroma(
        collection_name="company_info_store",
        persist_directory="chroma_storage",
        embedding_function=embeddings,
        collection_metadata={"hnsw:space": "cosine"}
    )

In [5]:
hash_id = "fa510296a7d556be18bf44d452698dc12c44e06edd0de9d3d5a2f81a5ca811f9"

all_ids = company_collection.get()["ids"]
if hash_id in all_ids:
    print(f"Hash ID '{hash_id}' exists in the collection.")
else:
    print(f"Hash ID '{hash_id}' does not exist in the collection.")


Hash ID 'fa510296a7d556be18bf44d452698dc12c44e06edd0de9d3d5a2f81a5ca811f9' does not exist in the collection.


In [ ]:
existing_docs = company_collection.get(where={"content_hash": content_hash})


In [2]:
from langchain_ollama import ChatOllama

In [ ]:
llm = ChatOllama(model="llama3.1",base_url="3.110.55.125:11434")
response=llm.invoke("hai who is this?")
print(response.content)
